# Code from Downey's _Think Python_

This notebook contains code for examples and exercises from chapter 4,
_Functions and Interfaces_ of Allen Downey's
_Think Python Third Edition_ (O'Reilly, 2024) using turtle graphics.

I removed the main text, keeping only the section titles to make it easier to locate the code in the book.
I also added some very brief comments for context.

## The Turtle module

Import the `Turtle` class:

In [1]:
from jupyturtle import Turtle

Create `Turtle` object, use a method:

In [2]:
t = Turtle()
t.forward(100)

Shortcuts for exploratory programming with an imperative API:

In [3]:
from jupyturtle import fd
fd(100)

The `fd()` call created a new turtle and canvas.
Further commands will update that same canvas.

Import more functions:

In [4]:
from jupyturtle import lt, rt, fd

Experiment with `left`, aliased to `lt`.

In [5]:
fd(50)
lt(90)
fd(50)

## Making a square

In [6]:
from jupyturtle import make_turtle, forward, left

make_turtle()

forward(50)
left(90)

forward(50)
left(90)

forward(50)
left(90)

forward(50)
left(90)

Using a `for` loop.

In [7]:
make_turtle()
for i in range(4):
    forward(50)
    left(90)

## Encapsulation and generalization

Write function

In [8]:
def square():
    for i in range(4):
        forward(50)
        left(90)

Call it.

In [9]:
make_turtle()
square()

Function with parameter

In [10]:
def square(length):
    for i in range(4):
        forward(length)
        left(90)

Now we can draw squares with different sizes.

In [11]:
make_turtle()
square(30)
square(60)

Generalization:

In [12]:
def polygon(n, length):
    angle = 360 / n
    for i in range(n):
        forward(length)
        left(angle)

Example:

In [13]:
make_turtle()
polygon(7, 30)

Named arguments:

In [14]:
make_turtle()
polygon(n=7, length=30)

## Approximating a circle

To draw a circle:

In [15]:
import math

def circle(radius):
    circumference = 2 * math.pi * radius
    n = 30
    length = circumference / n
    polygon(n, length)

The `delay` argument (named `speed` in ColabTurtle):

In [16]:
make_turtle(delay=0.05)
circle(30)

## Refactoring

`polyline`, a more general version of `polygon`:

In [17]:
def polyline(n, length, angle):
    for i in range(n):
        forward(length)
        left(angle)

Parameters:

`n`: number of segments to draw

`length`: length each segment

`angle`: angle between them

New `polygon` using `polyline`:

In [18]:
def polygon(n, length):
    angle = 360.0 / n
    polyline(n, length, angle)

Use `polyline` to write `arc`:

In [19]:
def arc(radius, angle):
    arc_length = 2 * math.pi * radius * angle / 360
    n = 30
    length = arc_length / n
    step_angle = float(angle) / n
    polyline(n, length, step_angle)

Rewrite `circle` to use `arc`:

In [20]:
def circle(radius):
    arc(radius,  360)

To check that these functions work as expected, we'll use them to draw something like a snail.

In [21]:
make_turtle(delay=0.01)
polygon(n=20, length=9)
arc(radius=70, angle=70)
circle(radius=10)

## Stack diagram

(no turtle examples in this section)

## Docstrings

`polyline` with docstring:

In [22]:
def polyline(n, length, angle):
    """Draws line segments with the given length and angle between them.

    n: integer number of line segments
    length: length of the line segments
    angle: angle between segments (in degrees)
    """
    for i in range(n):
        forward(length)
        left(angle)

In [23]:
make_turtle()
polyline(9, 30, 36)

## Debugging

(no turtle examples in this section)

## Glossary

(no turtle examples in this section)

## Exercises

`penup` and `pendown`

In [24]:
from jupyturtle import forward, penup, pendown

def jump(length):
    """Move forward length units without leaving a trail.

    Postcondition: Leaves the pen down.
    """
    penup()
    forward(length)
    pendown()

### Exercise

Draw a rectangle:

In [25]:
# Solution

def rectangle(length1, length2):
    """Draw a rectangle with the given lengths.

    length1: length of the first side
    length2: length of the second size
    """
    for i in range(2):
        forward(length1)
        left(90)
        forward(length2)
        left(90)

In [26]:
make_turtle()
rectangle(80, 40)

### Exercise

Draw a rhombus:

In [27]:
def rhombus(length, angle):
    for i in range(2):
        forward(length)
        left(angle)
        forward(length)
        left(180-angle)

In [28]:
make_turtle()
rhombus(50, 60)

### Exercise

Write `parallelogram`, then rewrite `rectangle` and `rhombus` to use it:

In [29]:
# Solution

def parallelogram(length1, length2, angle):
    for i in range(2):
        forward(length1)
        left(angle)
        forward(length2)
        left(180-angle)

In [30]:
# Solution

def rectangle(length1, length2):
    """Draw a rectangle with the given lengths.

    length1: length of the first side
    length2: length of the second size
    """
    parallelogram(length1, length2, 90)

In [31]:
# Solution

def rhombus(length, angle):
    parallelogram(length, length, angle)

In [32]:
make_turtle(delay=0.05)
jump(-120)

rectangle(80, 40)
jump(100)
rhombus(50, 60)
jump(80)
parallelogram(80, 50, 60)

### Exercise

A `triangle` function:


In [33]:
# Solution
import math

from jupyturtle import forward, left, right

def triangle(radius,  angle):
    """Draws an icosceles triangle.

    The turtle starts and ends at the peak, facing the middle of the base.

    radius: length of the equal legs
    angle: half peak angle in degrees
    """
    y = radius * math.sin(angle * math.pi / 180)

    right(angle)
    forward(radius)
    left(90+angle)
    forward(2*y)
    left(90+angle)
    forward(radius)
    left(180-angle)

(a little demonstration)

In [34]:
from jupyturtle import make_turtle, hide
make_turtle()
hide()

for i in range(12):
    triangle(50, 10)
    left(30)

In [35]:
# Solution

def draw_pie(n, radius):
    """Draws a pie divided into radial segments.

    n: number of segments
    radius: length of the radial spokes
    """
    angle = 360.0 / n
    for i in range(n):
        triangle(radius,  angle/2)
        left(angle)

In [36]:
make_turtle(delay=0.01)
draw_pie(5, 40)

In [37]:
make_turtle(delay=0.01)
jump(-120)

size = 40
draw_pie(5, size)
jump(2*size)
draw_pie(6, size)
jump(2*size)
draw_pie(7, size)
jump(2*size)
draw_pie(8, size)

### Exercise

Draw flowers with `petal`:

In [38]:
# Solution

def petal(radius,  angle):
    """Draws a petal using two arcs.

    t: Turtle
    radius: radius of the arcs
    angle: angle (degrees) that subtends the arcs
    """
    for i in range(2):
        arc(radius,  angle)
        left(180-angle)

In [39]:
# Solution

def flower(n, radius, angle):
    """Draws a flower with n petals.

    n: number of petals
    radius: radius of the arcs
    angle: angle (degrees) that subtends the arcs
    """
    for i in range(n):
        petal(radius,  angle)
        left(360/n)

In [40]:
make_turtle()

jump(-100)
flower(7, 60.0, 60.0)

jump(100)
flower(10, 40.0, 80.0)

In [41]:
# this is drawing has lots of sides, so it may be very slow or even crash you browser

make_turtle() # start a new canvas
hide() # don't draw the turtle
jump(100)
flower(14, 140.0, 20.0)

### Ask an assistant

Draw a spiral:

In [42]:
# Solution

from jupyturtle import make_turtle, forward, right

make_turtle()

def spiral(length, angle):
    for _ in range(100):
        forward(length)
        right(angle)
        length += 2  # Increase the length for each segment

spiral(5, 90)

In [43]:
# Solution

# prompt: make that a circular spiral, and don't change the name of the module

from jupyturtle import make_turtle, forward, left
import math

make_turtle()

def circular_spiral(radius, angle):
    rotations = 5
    distance = 2 * math.pi * radius / 360  # Calculate the distance for each degree of rotation
    for _ in range(rotations * 360):
        forward(distance)
        left(angle)
        radius += 0.1  # Increase the radius for each segment
        distance = 2 * math.pi * radius / 360  # Recalculate the distance for the updated radius

circular_spiral(50, 1)
print('done')

done
